## 1. Installing packages needed

In [83]:
!pip install mne
!pip install yasa

You should consider upgrading via the '/Users/richardpang/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [149]:
import os
import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
import xml.etree.ElementTree
from xml.etree.ElementTree import parse
from scipy import signal
import seaborn as sns
%matplotlib qt

## 2. Data Loading
#### The raw data is an edf file downloaded from the EEG Motor Movement/Imagery Dataset. You may find the dataset here: https://www.physionet.org/content/eegmmidb/1.0.0/. Noted that it contains 64 channels and the sampling frequency is 160 Hz as indicated below. 

In [220]:
raw = mne.io.read_raw_edf('data/S001R01.edf')
raw.pick_types(eeg=True)
raw.plot(start=20,duration=1)
plt.show()

Extracting EDF parameters from /Users/richardpang/Desktop/eeg_analysis/data/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Opening raw-browser...
Closing raw-browser...
Channels marked as bad:
none


In [221]:
# Extract the data and convert from V to uV
data = raw.get_data(units="uV")
sf = raw.info['sfreq']
chan = raw.ch_names

# Let's have a look at the data
print('Chan =', chan)
print('Sampling frequency =', sf, 'Hz')
print('Data shape =', data.shape)

Chan = ['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..']
Sampling frequency = 160.0 Hz
Data shape = (64, 9760)


In [219]:
print('The actual data is just a matrix array!\n\n {}\n'.format(raw.get_data()))

The actual data is just a matrix array!

 [[-1.6e-05 -5.6e-05 -5.5e-05 ...  0.0e+00  0.0e+00  0.0e+00]
 [-2.9e-05 -5.4e-05 -5.5e-05 ...  0.0e+00  0.0e+00  0.0e+00]
 [ 2.0e-06 -2.7e-05 -2.9e-05 ...  0.0e+00  0.0e+00  0.0e+00]
 ...
 [-2.1e-05 -1.2e-05  2.0e-06 ...  0.0e+00  0.0e+00  0.0e+00]
 [-1.1e-05  1.0e-06  1.8e-05 ...  0.0e+00  0.0e+00  0.0e+00]
 [ 1.5e-05  2.1e-05  3.5e-05 ...  0.0e+00  0.0e+00  0.0e+00]]



In [209]:
raw.info

<Info | 7 non-empty values
 bads: []
 ch_names: Fc5., Fc3., Fc1., Fcz., Fc2., Fc4., Fc6., C5.., C3.., C1.., ...
 chs: 64 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
>

## 3. Plot the PSD

In [232]:
from scipy.signal import welch

win = int(4 * sf)  # Window size is set to 4 seconds
freqs, psd = welch(data, sf, nperseg=win, average='median')  # Works with single or multi-channel data

print(freqs.shape, psd.shape)  # psd has shape (n_channels, n_frequencies)

# Plot
plt.plot(freqs, psd[1], 'k', lw=2)
plt.fill_between(freqs, psd[1], cmap='Spectral')
plt.xlim(0, 80)
plt.yscale('log')
sns.despine()
plt.title(chan[1])
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD log($uV^2$/Hz)');

(321,) (64, 321)


## 4.1 Calculate band powers from PSD (relative and absolute)

In [233]:
yasa.bandpower_from_psd(psd, freqs, ch_names=chan)

,Chan,Delta,Theta,Alpha,Sigma,Beta,Gamma,TotalAbsPow,FreqRes,Relative
0,Fc5.,0.546720,0.188663,0.081719,0.071354,0.086720,0.024824,1333.560137,0.25,True
1,Fc3.,0.553381,0.186151,0.088004,0.079254,0.075679,0.017531,1591.182522,0.25,True
2,Fc1.,0.577314,0.180922,0.088535,0.069158,0.068330,0.015740,1711.810271,0.25,True
3,Fcz.,0.625901,0.160527,0.073067,0.059309,0.067279,0.013917,1837.897807,0.25,True
4,Fc2.,0.638252,0.153374,0.070831,0.061820,0.061709,0.014013,1706.774518,0.25,True
...,...,...,...,...,...,...,...,...,...,...
59,Po8.,0.611308,0.128695,0.089872,0.074769,0.075307,0.020049,1463.046097,0.25,True
60,O1..,0.551645,0.133374,0.107755,0.091970,0.102906,0.012351,1852.322341,0.25,True
61,Oz..,0.590588,0.124811,0.089765,0.078538,0.100867,0.015432,1840.485750,0.25,True
62,O2..,0.616492,0.120480,0.085481,0.067933,0.093260,0.016354,1992.397458,0.25,True


In [234]:
yasa.bandpower_from_psd(psd, freqs, ch_names=chan,relative=False)

,Chan,Delta,Theta,Alpha,Sigma,Beta,Gamma,TotalAbsPow,FreqRes,Relative
0,Fc5.,729.083449,251.593684,108.977588,95.155092,115.646269,33.104056,1333.560137,0.25,False
1,Fc3.,880.530178,296.200884,140.030286,126.106843,120.418670,27.895661,1591.182522,0.25,False
2,Fc1.,988.252429,309.703739,151.555804,118.386149,116.968244,26.943906,1711.810271,0.25,False
3,Fcz.,1150.341449,295.031470,134.289831,109.004222,123.652382,25.578453,1837.897807,0.25,False
4,Fc2.,1089.352459,261.774983,120.893042,105.512753,105.323430,23.917851,1706.774518,0.25,False
...,...,...,...,...,...,...,...,...,...,...
59,Po8.,894.372006,188.287211,131.487208,109.390743,110.176915,29.332015,1463.046097,0.25,False
60,O1..,1021.823602,247.050760,199.597288,170.357692,190.614198,22.878800,1852.322341,0.25,False
61,Oz..,1086.968179,229.712791,165.211522,144.547493,185.643865,28.401902,1840.485750,0.25,False
62,O2..,1228.297022,240.044884,170.311805,135.349890,185.810987,32.582869,1992.397458,0.25,False


## 4.2 Calculate band powers from raw data (relative and absolute)

In [230]:
# Relative bandpower per channel on the whole recording (entire data)
yasa.bandpower(data, sf=sf, ch_names=chan)

,Delta,Theta,Alpha,Sigma,Beta,Gamma,TotalAbsPow,FreqRes,Relative
Chan,,,,,,,,,
Fc5.,0.548339,0.188002,0.085510,0.069651,0.084315,0.024183,1356.084169,0.25,True
Fc3.,0.569570,0.174826,0.087572,0.076244,0.074501,0.017287,1627.558176,0.25,True
Fc1.,0.581620,0.177363,0.086579,0.070425,0.068284,0.015729,1695.851253,0.25,True
Fcz.,0.611424,0.169222,0.074378,0.062707,0.068008,0.014262,1821.866101,0.25,True
Fc2.,0.634056,0.153610,0.074786,0.061335,0.061986,0.014227,1689.180796,0.25,True
...,...,...,...,...,...,...,...,...,...
Po8.,0.611426,0.127151,0.089966,0.074720,0.076852,0.019886,1461.909318,0.25,True
O1..,0.555521,0.130322,0.107836,0.090734,0.103069,0.012518,1858.856100,0.25,True
Oz..,0.588168,0.122236,0.092429,0.078167,0.103260,0.015741,1828.575233,0.25,True


In [231]:
yasa.bandpower(raw,relative=False)

,Delta,Theta,Alpha,Sigma,Beta,Gamma,TotalAbsPow,FreqRes,Relative
Chan,,,,,,,,,
Fc5.,743.594341,254.945948,115.958546,94.453281,114.337948,32.794104,1356.084169,0.25,False
Fc3.,927.008354,284.540254,142.528072,124.091109,121.254069,28.136317,1627.558176,0.25,False
Fc1.,986.341575,300.781044,146.824816,119.430047,115.800208,26.673562,1695.851253,0.25,False
Fcz.,1113.932683,308.299800,135.506160,114.242997,123.900846,25.983616,1821.866101,0.25,False
Fc2.,1071.034496,259.475381,126.327908,103.606599,104.704983,24.031429,1689.180796,0.25,False
...,...,...,...,...,...,...,...,...,...
Po8.,893.849195,185.882762,131.521556,109.234149,112.350669,29.070987,1461.909318,0.25,False
O1..,1032.634114,242.250511,200.450749,168.661942,191.589984,23.268800,1858.856100,0.25,False
Oz..,1075.508674,223.517801,169.012913,142.934222,188.818328,28.783294,1828.575233,0.25,False


### 4.3 Note that there are slight differences between the results calculated from PSD and the raw data. It is because there is a tiny loss during the calculation from PSD. Also, the results obtained from different approaches (e.g. the welch approach and the Multitaper approach)